# Import packages and initialize environment

In [ ]:
from collections import defaultdict
from typing import List, Tuple
import csv
import tqdm
from copy import deepcopy
from math import log, isclose

# Prepare the tokenized parallel corpus

In [ ]:
parallel_tokens_path = './cambridge_parallel_tokens.tsv'
parallel_tokens = []
with open(parallel_tokens_path, 'r', encoding='utf8') as f:
    for en_tokens, ch_tokens in csv.reader(f, delimiter='\t'):
        parallel_tokens.append([en_tokens.split('#sep#'),
                                ch_tokens.split('#sep#')])

In [ ]:
for x in parallel_tokens[:3]:
    print(x)

# IBM Model 1
According to the equations in the assignment instructions, if we want to update the translation probablity of a english word $e$ given a foreign word $f$, we have to count how often $e$ aligned with $f$ in the whole parallel corpus. Lets implement the function to count that in a pair of sentences.
$$c(e|f;\mathbf E, \mathbf F) = \frac{t(e|f)}{\sum_{i=0}^{l_F}t(e|f_i)}\sum_{j=1}^{l_E}\delta(e, e_j)\sum_{i=0}^{l_F}\delta(f, f_i) \\
 \delta(x, y) = \begin{array}{lr}1,\quad if \quad x == y \\ 0, \quad else \end{array}$$
Note that the word at index 0 of foreign sentence $\mathbf F$ is NULL token.  
In the next two steps, you will implement this count function.

## Step 1. Compute the normalization term of the count function
In the first step, the returned variable of your function, ```total_e```, should represent $\sum_{i=0}^{l_F}t(e|f_i)$.  
Note that ```total_e``` is the denominator of $c$.  
Here is the example of the usage of it :  
```total_e['apple']```$= \sum_{i=0}^{l_F}t(apple|f_i)$

In [ ]:
def compute_normalization_term(en_token_list: List[str], ch_token_list: List[str],
                               t_ef: defaultdict) -> defaultdict:
    total_e = defaultdict(lambda: 0)
    #### YOUR CODE HERE ####

    return total_e

In [ ]:
example_ch = ['[NULL]', '我', '愛', '你', '愛', '我']
example_en = ['i', 'love', 'that', 'you', 'love', 'me']


""" result
('[NULL]', 0.6)
('我', 1.2)
('愛', 1.2)
('你', 0.6)
"""
for x in compute_normalization_term(example_en, example_ch,
                                    defaultdict(lambda: defaultdict(lambda: 0.1))).items():
    print(x)

## Step 2. Collect the count from the given bilingual sentences
In the second step, the returned variable of your function, ```count_of_sent```, should represent the complete count function.  
  Here is the example of the usage of it :  
  ```count_of_sent['apple']['蘋果']``` $= c(apple|蘋果;\mathbf E, \mathbf F)$

In [ ]:
def collect_ef_count(en_token_list: List[str], ch_token_list: List[str],
                     t_ef: defaultdict, total_e: defaultdict) -> defaultdict:
    count_of_sent = defaultdict(lambda: defaultdict(lambda: 0.0))
    #### YOUR CODE HERE ####

    return count_of_sent

In [ ]:
print(example_ch, example_en)
print()

"""result
('i', '[NULL]', 0.01)
('i', '我', 0.02)
('i', '愛', 0.02)
('i', '你', 0.01)
('love', '[NULL]', 0.02)
('love', '我', 0.04)
('love', '愛', 0.04)
('love', '你', 0.02)
('that', '[NULL]', 0.01)
('that', '我', 0.02)
('that', '愛', 0.02)
('that', '你', 0.01)
('you', '[NULL]', 0.01)
('you', '我', 0.02)
('you', '愛', 0.02)
('you', '你', 0.01)
('me', '[NULL]', 0.01)
('me', '我', 0.02)
('me', '愛', 0.02)
('me', '你', 0.01)
"""
for x, d in collect_ef_count(example_en, example_ch, 
                             defaultdict(lambda: defaultdict(lambda: 0.1)),
                             defaultdict(lambda: 10.0)).items():
    for y, z in d.items():
        print((x, y, z))

## Step 3. Implement IBM Model 1
Now you should be able to re-estimate the lexical probability distribution by
$$t(e|f) = \frac{\sum_{(\mathbf E, \mathbf F)}c(e|f;\mathbf E, \mathbf F)}{\sum_{e'}\sum_{(\mathbf e, \mathbf f)}c(e'|f;\mathbf E, \mathbf F)}$$
Note: You need to fill the code into where the comment strings start with '####'.

In [ ]:
def ibm_model_1(parallel_corpus_list: List, n_iters: int) -> defaultdict:
    en_vocab = set(en_token.lower() for en_token_list, _ in parallel_corpus_list
                            for en_token in en_token_list)
    ch_vocab = set(ch_token for _, ch_token_list in parallel_corpus_list
                            for ch_token in ch_token_list)
    print(f'length of en_vocab: {len(en_vocab)}')
    print(f'lenght of ch_vocab: {len(ch_vocab)}')
    
    null_token = '[NULL]'
    # Add the Null token
    ch_vocab.add(null_token)
    
    # ptotal_edistribution is uniform.
    init_prob = 1 / len(en_vocab)
    # t_ef[e][f] means t(e|f)
    t_ef = defaultdict(lambda: defaultdict(lambda: init_prob))
    
    
    # EM process
    for i in tqdm.tqdm(range(n_iters)):
        # initialize
        # total_ef_count[e][f] is the numerator ot new t(e|f)
        total_ef_count = defaultdict(lambda: defaultdict(lambda: 0.0))
        # total_f[f] is the Denominator ot new t(e|f)
        total_f = defaultdict(lambda: 0.0) 
        token_pair_set = set()
        perp = 0
        # collect the evidence from corpus
        for en_token_list, ch_token_list in parallel_corpus_list:
            ch_token_list = [null_token] + ch_token_list
            perp += compute_perp(en_token_list, ch_token_list, t_ef)
            
            total_e = compute_normalization_term(en_token_list, ch_token_list, t_ef)
            ef_count_of_sent = collect_ef_count(en_token_list, ch_token_list, t_ef, total_e)
            
            cur_token_pair_set = set((e, f) for e in en_token_list
                                            for f in ch_token_list)
            for e, f in cur_token_pair_set:
                #### You should write two lines of code here.  ####
                #### Update total_ef_count and total_f with ef_count_of_sent ####

            token_pair_set.update(cur_token_pair_set)
            
        print(f'perplexity: {round(perp, 2)}')
        # estimate probabilities
        t_ef = defaultdict(lambda: defaultdict(lambda: 0))
        for e, f in token_pair_set:
            #### You should write a line of code here. Update t_ef ####
        
    return t_ef

def compute_perp(en_token_list, ch_token_list, t_ef):
    l_e = len(en_token_list)
    l_f = len(ch_token_list)
    p_EF = 0.
    for e in en_token_list:
        s = 0.
        for f in ch_token_list:
            s += t_ef[e][f]
        p_EF += -log(s)
    p_EF += log(l_f**l_e)
    return p_EF

# Training
**You have to run the following cells.**

In [ ]:
t_ef = ibm_model_1(parallel_tokens, 10)

# Estimation
**You have to run the following cells.**

In [ ]:
def get_conditional_prob(token_ch, translation_dict):
    prob_dict = {}
    for token_en, sub_dict in translation_dict.items():
        if token_ch in sub_dict:
            prob_dict[token_en] = sub_dict[token_ch]
    return prob_dict

def check_top_k_prob(token_ch, k, translation_dict):
    prob_dict = get_conditional_prob(token_ch, translation_dict)
    prob_list = list(prob_dict.items())
    prob_sum = sum(prob_dict.values())
    prob_list.sort(key=lambda x:x[1], reverse=True)
    
    if not isclose(prob_sum, 1.):
        print(f"Warning. sum of p( e | {token_ch}) = {prob_sum}, not close to 1.")
    
    for token_en, prob in prob_list[:k]:
        print(f"p({token_en} | {token_ch}) = {prob}")

In [ ]:
""" results for reference
p(school | 學校) = 0.7592438820372703
p(schools | 學校) = 0.152040212014728
p(at | 學校) = 0.08116603608007476
p(the | 學校) = 0.0024247348456308397
p(has | 學校) = 0.0024026143118947804
"""

check_top_k_prob('學校', 5, t_ef)

In [ ]:
""" results for reference
p(businesses | 企業) = 0.46758578752806945
p(business | 企業) = 0.3901655681518314
p(by | 企業) = 0.03300134949576897
p(run | 企業) = 0.02862452153472591
p(a | 企業) = 0.019347178868170722
"""

check_top_k_prob('企業', 5, t_ef)

In [ ]:
""" results for reference
p(budget | 預算) = 0.675000585629638
p(are | 預算) = 0.1145673270578127
p(within | 預算) = 0.03939258531542436
p(already | 預算) = 0.03737763058988209
p(overspent | 預算) = 0.024970159365872654
"""

check_top_k_prob('預算', 5, t_ef)

In [ ]:
check_top_k_prob('世紀', 5, t_ef)

In [ ]:
check_top_k_prob('紡織', 5, t_ef)

In [ ]:
check_top_k_prob('校長', 5, t_ef)

In [ ]:
check_top_k_prob('美國', 5, t_ef)

In [ ]:
check_top_k_prob('員警', 5, t_ef)

# IBM Model 2
Recall that the count functions of lexical translation and alignment in the assignment instructions:
$$\begin{eqnarray}
c_1(e|f;\mathbf E, \mathbf F) 
&=&\sum_{j=1}^{l_E}\sum_{i=0}^{l_F}
\frac{t(e|f)a(i|j,l_E,l_F )\delta(e, e_j)\delta(f, f_i)}
{\sum_{i'=0}^{l_F}t(e|f_{i'})a(i'|j,l_E,l_F )}
\end{eqnarray}$$

$$\begin{eqnarray}
c_2(i|j, l_E, l_F;\mathbf E, \mathbf F) &=& 
\frac{t(e_j|f_i) a(i|j,l_E,l_F)}{\sum_{i'=0}^{l_F}t(e_j|f_{i'})a(i'|j,l_E,l_F )}
\end{eqnarray}$$

Similarly, you will implement the count functions in two steps. In the final step, you would be asked to complete the implementation of IBM Model 2.

## Step 1. Compute the normalization term of the count function
```total_e[english_word][j]``` is a defaultdict with two keys, and it is actually the count function of alignment.  
You may be aware to the fact that $\sum_{j}$```total_e[english_word][j]``` is the count function of lexical translation if you note the difference between the normalization terms of them. 

In [ ]:
def compute_alg_normalization_term(en_token_list: List[str], ch_token_list: List[str],
                                   t_ef: defaultdict, q_alg: defaultdict) -> defaultdict:
    total_e = defaultdict(lambda: defaultdict(lambda: 0))
    l_e = len(en_token_list)
    # the lenth of foreign sentence does not take NULL token into account
    l_f = len(ch_token_list) - 1
    
    for j in range(1, l_e + 1):
        en_word = en_token_list[j - 1]
        for i in range(l_f + 1):
            ch_word = ch_token_list[i]
            #### YOUR CODE HERE ####

    return total_e

In [ ]:
print(example_ch, example_en)
print()

""" result
('i', 1, 0.12000000000000002)
('love', 2, 0.12000000000000002)
('love', 5, 0.12000000000000002)
('that', 3, 0.12000000000000002)
('you', 4, 0.12000000000000002)
('me', 6, 0.12000000000000002)
"""
for x, d in compute_alg_normalization_term(example_en, example_ch,
                                           defaultdict(lambda: defaultdict(lambda: 0.1)),
                                           defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0.2))))
                                          ).items():
    for y, z in d.items():
        print((x, y, z))

## Step 2. Collect the count for translation and the count for alignment from the given bilingual sentences

In [ ]:
def collect_count(en_token_list: List[str], ch_token_list: List[str],
                    t_ef: defaultdict, q_alg: defaultdict,
                    total_e: defaultdict) -> defaultdict:
    ef_count = defaultdict(lambda: defaultdict(lambda: 0))
    alg_count = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0.0))))
    l_e = len(en_token_list)
    # the lenth of foreign sentence does not take NULL token into account
    l_f = len(ch_token_list) - 1
    
    for j in range(1, l_e + 1):
        en_word = en_token_list[j - 1]
        for i in range(l_f + 1):
            ch_word = ch_token_list[i]
            #### YOUR CODE HERE ####
            
            
    return (ef_count, alg_count)

In [ ]:
print(example_ch, example_en)
print()

"""result
('i', '[NULL]', 0.20000000000000004)
('i', '我', 0.4000000000000001)
('i', '愛', 0.4000000000000001)
('i', '你', 0.20000000000000004)
('love', '[NULL]', 0.4000000000000001)
('love', '我', 0.8000000000000002)
('love', '愛', 0.8000000000000002)
('love', '你', 0.4000000000000001)
('that', '[NULL]', 0.20000000000000004)
('that', '我', 0.4000000000000001)
('that', '愛', 0.4000000000000001)
('that', '你', 0.20000000000000004)
('you', '[NULL]', 0.20000000000000004)
('you', '我', 0.4000000000000001)
('you', '愛', 0.4000000000000001)
('you', '你', 0.20000000000000004)
('me', '[NULL]', 0.20000000000000004)
('me', '我', 0.4000000000000001)
('me', '愛', 0.4000000000000001)
('me', '你', 0.20000000000000004)

(0, 1, 0.20000000000000004)
...
(5, 5, 0.20000000000000004)
(5, 6, 0.20000000000000004)
"""
example_ef, example_alg = collect_count(example_en, example_ch,
                                        defaultdict(lambda: defaultdict(lambda: 0.1)),
                                        defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0.2)))),
                                        defaultdict(lambda: defaultdict(lambda: 0.1))
                                       )
for x, d in example_ef.items():
    for y, z in d.items():
        print((x, y, z))
print()

for x, d in example_alg.items():
    for y, z in d.items():
        print((x, y, z[len(example_en)][len(example_ch)-1]))
print()

## Step3. Implement IBM Model 2
Now you should be able to re-estimate the probability distribution of lexical translation and that of alignment by  
$$t(e|f) = \frac{\sum_{(\mathbf E, \mathbf F)}c_1(e|f;\mathbf E, \mathbf F)}{\sum_{e'}\sum_{(\mathbf E, \mathbf F)}c_1(e'|f;\mathbf E, \mathbf F)}$$

$$a(i|j, l_E, l_F) = \frac{\sum_{(\mathbf E, \mathbf F)}c_2(i|j, l_E, l_F;\mathbf E, \mathbf F)}{\sum_{i'}\sum_{(\mathbf E, \mathbf F)}c_2(i'|j, l_E, l_F;\mathbf E, \mathbf F)}$$

Note: You need to fill the code into where the comment strings start with '####'.

In [ ]:
def ibm_model_2(parallel_corpus_list: List, n_iters: int,
                init_t_ef: defaultdict) -> Tuple[defaultdict, defaultdict]:
    en_vocab = set(en_token.lower() for en_token_list, _ in parallel_corpus_list
                            for en_token in en_token_list)
    ch_vocab = set(ch_token for _, ch_token_list in parallel_corpus_list
                            for ch_token in ch_token_list)
    print(f'length of en_vocab: {len(en_vocab)}')
    print(f'lenght of ch_vocab: {len(ch_vocab)}')
    null_token = '[NULL]'
    # Add the Null token
    ch_vocab.add(null_token)
    
    t_ef = deepcopy(init_t_ef)
    q_alg = get_initial_q_alg(parallel_corpus_list) # q_alg[i][j][l_e][l_f] means a(i | j, l_e, l_f)
    
    for i in tqdm.tqdm(range(n_iters)):
        total_ef_count = defaultdict(lambda: defaultdict(lambda: 0.0))
        total_f = defaultdict(lambda: 0.0)

        # total_alg_count[i][j][l_e][l_f] is the numerator of new a(i | j, l_e, l_f)
        total_alg_count = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0.0))))
        # total_alg_count[j][l_e][l_f] is the denominator of new a(i | j, l_e, l_f)
        total_alg = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0.0)))

        token_pair_set = set()
        perp = 0
        # collect the evidence from corpus
        for en_token_list, ch_token_list in parallel_corpus_list:
            ch_token_list = [null_token] + ch_token_list
            perp += compute_perp(en_token_list, ch_token_list, t_ef, q_alg)
            # the lenth of foreign sentence does not take NULL token into account
            l_f = len(ch_token_list) - 1
            l_e = len(en_token_list)

            total_e = compute_alg_normalization_term(en_token_list, ch_token_list, t_ef, q_alg)
            ef_count_of_sent, alg_count_of_sent = collect_count(en_token_list, ch_token_list,
                                                                t_ef, q_alg, total_e)
            
            cur_token_pair_set = set((e, f) for e in en_token_list
                                            for f in ch_token_list)
            token_pair_set.update(cur_token_pair_set)
            for e, f in cur_token_pair_set:
                #### You should write two lines of code here.  ####
                #### Update total_ef_count and total_f with ef_count_of_sent####
                
                
            # collect counts
            for j in range(1, l_e+1):
                for i in range(l_f+1):
                    #### You should write two lines of code here.  ####
                    #### Update total_alg_count, total_alg with alg_count_of_sent ####
                    
                    
        print(f'perplexity: {round(perp, 2)}')
        
        # Estimate the new lexical translation probabilities
        t_ef = defaultdict(lambda: defaultdict(lambda: 0))
        for e, f in token_pair_set:
            #### You should write a line of code here. Update t_ef ####
            
            
        # Estimate the new alignment probabilities
        q_alg = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0.0))))
        for en_token_list, ch_token_list in parallel_corpus_list:
            l_f = len(ch_token_list)
            l_e = len(en_token_list)
            for i in range(l_f + 1):
                for j in range(1, l_e + 1):
                    #### You should write a line of code here. Update q_alg ####
                    
    return (t_ef, q_alg)

def get_initial_q_alg(parallel_corpus_list: List) -> defaultdict:
    q_alg = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0.0))))
    
    for en_token_list, ch_token_list in parallel_corpus_list:
        l_e = len(en_token_list)
        l_f = len(ch_token_list)
        q_initial = 1 / (l_f + 1)
        for i in range(0, l_f + 1): # 0 for NULL token
            for j in range(1, l_e + 1):
                q_alg[i][j][l_e][l_f] = q_initial
    return q_alg

def compute_perp(en_token_list, ch_token_list, t_ef, q_alg):
    perp = 0
    l_e = len(en_token_list)
    l_f = len(ch_token_list) - 1
    for j in range(1, l_e+1):
        en_word = en_token_list[j - 1]
        cur_perp = 0
        for i in range(l_f + 1):
            ch_word = ch_token_list[i]
            cur_perp += t_ef[en_word][ch_word] * q_alg[i][j][l_e][l_f]
        perp += -log(cur_perp)
    return perp

# Estimation
**You have to run the following cells.**

In [ ]:
t_ef_2, q_alg = ibm_model_2(parallel_tokens, 15, t_ef)

In [ ]:
def get_top_k_prob(token_ch, k, translation_dict):
    prob_dict = get_conditional_prob(token_ch, translation_dict)
    prob_list = list(prob_dict.items())
    prob_sum = sum(prob_dict.values())
    prob_list.sort(key=lambda x:x[1], reverse=True)
    if not isclose(prob_sum, 1.):
        print(f"Warning. sum of p( e | {token_ch}) = {prob_sum}, not close to 1.")
    
    return prob_list[:k]

def translate(ch_token_list: List[str], l_e: int, translation_dict, align):
    l_f = len(ch_token_list)
    null_token = '[NULL]'
    ch_token_list.insert(0, null_token)
    k = 3
    
    en_candidates = []
    for token_ch in ch_token_list:
        en_candidates.append(get_top_k_prob(token_ch, 10, translation_dict))

    for j in range(1, l_e+1):
        possible_list = []
        for i, ens in enumerate(en_candidates):
            for token_en, prob in ens:
                possible_list.append((token_en, align[i][j][l_e][l_f] * prob))
        possible_list.sort(key=lambda x:x[1], reverse=True)
        print(f"possible candidates for the {j}-th English word:")
        for token_en, prob in possible_list[:k]:
            print(f"{token_en}\t\tprobability:{prob}")
        print()


In [ ]:
""" result
possible candidates for the 1-th word:
i		probability:0.3051610136893576
a		probability:0.08537920692347092
(		probability:0.07314717156010928

possible candidates for the 2-th word:
i		probability:0.2220627843510443
bike		probability:0.1721509922781537
bicycle		probability:0.07748032610499858

possible candidates for the 3-th word:
riding		probability:0.08936528673455782
bike		probability:0.08840401330382662
will		probability:0.07680236346367117

possible candidates for the 4-th word:
i		probability:0.15463795029280586
riding		probability:0.08968372901915127
will		probability:0.07672989741828276

possible candidates for the 5-th word:
bike		probability:0.3455995109696529
bicycle		probability:0.15554463240264935
i		probability:0.08811550005889636
"""
translate(['我', '會', '騎', '腳踏車'], 5, t_ef_2, q_alg)

In [ ]:
translate(['今天','天氣','很','好'], 5, t_ef_2, q_alg)